In [ ]:
!pip install yt-dlp spleeter -q

In [2]:
import os
import re
import subprocess
import contextlib
import logging
from functools import wraps
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import librosa
import yt_dlp
from spleeter.separator import Separator
import ffmpeg

import torch
import torchaudio
from transformers import AutoModelForAudioClassification, ASTFeatureExtractor

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def suppress_output(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Redirect stdout and logging to os.devnull
        with open(os.devnull, 'w') as fnull:
            with contextlib.redirect_stdout(fnull), contextlib.redirect_stderr(fnull):
                # Configure logging to use a NullHandler
                null_handler = logging.NullHandler()
                root_logger = logging.getLogger()
                original_handlers = root_logger.handlers.copy()
                root_logger.handlers = [null_handler]
                try:
                    result = func(*args, **kwargs)
                finally:
                    # Restore the original logging handlers
                    root_logger.handlers = original_handlers
        return result
    return wrapper

In [23]:
#@suppress_output
def download_youtube_audio(url):
    ydl_opts = {
        'format': 'bestaudio',
        'format_sort': ['+size'],
        'outtmpl': '%(id)s.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',  
        }]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
        video_info = ydl.extract_info(url, download=False)
    
    video_path_local = Path(f"{video_info['id']}.wav")
    
    return video_path_local

In [6]:
@suppress_output
def slice_wav_by_length(input_file, outfile_name_pattern, desired_length=600):
    # FFmpeg command to split the WAV file into 10-minute segments
    command = [
        'ffmpeg',
        '-hide_banner',  # Added to hide FFmpeg banner
        '-loglevel', 'error',  # Set log level to error
        '-i', input_file,
        '-f', 'segment',
        '-segment_time', str(desired_length),
        '-c', 'copy',
        f"{outfile_name_pattern}%03d.wav"
    ]

    # Run the FFmpeg command
    subprocess.run(command)

In [7]:
@suppress_output
def seperate_vocal_and_accompanies(separator, output_paths):
    output_directory = 'output'
    os.makedirs(output_directory, exist_ok=True)
    
    for file_path in tqdm(output_paths, desc="Processing files"):
        output_file_path = os.path.join(output_directory, os.path.basename(file_path))
        separator.separate_to_file(file_path, output_file_path)

In [8]:
def get_filepaths_with_string_and_extension(
    root_directory='.', target_string='', extension=''):
    return sorted([os.path.abspath(os.path.join(root, file)) 
         for root, _, files in os.walk(root_directory)
         for file in files 
         if target_string in file and (not extension or file.endswith(extension))]
    )

In [9]:
def load_and_slice_audio(vocal_paths, slice_duration=10):
    all_samples = []
    all_sampling_rates = []

    for path in tqdm(vocal_paths):
        waveform, sampling_rate = torchaudio.load(path)
        waveform = waveform.squeeze().numpy()

        num_slices = waveform.shape[-1] // (sampling_rate * slice_duration)

        samples = [
            torch.from_numpy(
                waveform[:, i * sampling_rate * slice_duration : (i + 1) * sampling_rate * slice_duration]
            )
            for i in range(num_slices)
        ]

        all_samples.extend(samples)
        all_sampling_rates.extend([sampling_rate] * num_slices)

    return all_samples, all_sampling_rates

In [10]:
def get_common_sampling_rate(sampling_rates):
    if len(set(sampling_rates)) == 1:
        return int(sampling_rates[0])
    else:
        raise ValueError("All sampling rates should be the same.")

In [11]:
def predict_samples(model, feature_extractor, monofied_samples, sampling_rate, device):
    result = []
    raw_outputs = []

    for i, sample in tqdm(enumerate(monofied_samples), total=len(monofied_samples), desc="Processing samples"):
        inputs = feature_extractor(
            sample, feature_size=2, sampling_rate=sampling_rate, 
            padding="max_length", return_tensors="pt"
        )
        
        input_values = inputs.input_values.to(device)

        with torch.no_grad():
            outputs = model(input_values)
            predicted_labels = model.config.id2label[outputs.logits.argmax(-1).item()]
            result.append([i, predicted_labels])
            raw_outputs.append(outputs)

    return result, raw_outputs

In [135]:
def create_dataframe(predictions, time_window_length):
    df = pd.DataFrame(predictions, columns=['index', 'class'])
    df['start'] = df['index'].apply(lambda x: seconds_to_hhmmss(x*time_window_length))
    df['end'] = df['index'].apply(lambda x: seconds_to_hhmmss((x+1)*time_window_length))
    return df

In [13]:
def audio_analyzer(file_paths, model, device, slice_duration):
    sliced_samples, sampling_rates = load_and_slice_audio(file_paths, slice_duration)
    monofied_samples = [torch.mean(x, dim=0) for x in sliced_samples]
    sampling_rate = get_common_sampling_rate(sampling_rates)

    feature_extractor = ASTFeatureExtractor(
        sampling_rate=sampling_rate, do_normalize=True
    )

    predictions, raw_outputs = predict_samples(
        model, feature_extractor, monofied_samples, sampling_rate, device
    )

    result_df = create_dataframe(predictions, time_window_length=slice_duration)

    return result_df, raw_outputs

In [14]:
def class_manipulator_old(input_df):
    # Create a copy of the input DataFrame to avoid modifying the original
    df = input_df.copy()

    # Binarize the 'class' column
    df['binarized_class'] = df['class'].apply(
        lambda x: 0 if (x in ['Speech','Tick', 'Clicking', ]) else 1
    )

    # Create the 'group' column
    df['group'] = (df['binarized_class'] != df['binarized_class'].shift()).cumsum()

    # Group the DataFrame by the new 'group' column
    grouped_df = df.groupby(['binarized_class', 'group']).agg(
        start=('start', 'first'),
        end=('end', 'last'),
        class_value=('class', 'first')
    ).reset_index(drop=True)

    # Filter out unwanted classes and sort by 'start'
    intermediate_df = grouped_df[
        ~grouped_df['class_value'].isin(['Speech','Tick', 'Clicking',])
    ].sort_values('start').reset_index(drop=True)

    return intermediate_df

In [15]:
def class_manipulator(input_df, singing_like_classes):
    
    def _get_most_frequent_with_fallback(x):
        modes = x.mode()
        if not modes.empty:
            if any(mode in singing_like_classes for mode in modes):
                return 'Singing'
        return None #'Speech'
    
    # Create a copy of the input DataFrame to avoid modifying the original
    df = input_df.copy()
    
    # Binarize the 'class' column
    df['binarized_class'] = df['class'].apply(
        lambda x : 1 if (x in singing_like_classes) else 0
    )

    # Create the 'group' column
    df['group'] = (df['binarized_class'] != df['binarized_class'].shift()).cumsum()

    # Group the DataFrame by the new 'group' column
    grouped_df = df.groupby(['binarized_class', 'group']).agg(
        start=('start', 'first'),
        end=('end', 'last'),
        class_value=('class', _get_most_frequent_with_fallback) 
    ).reset_index(drop=True).dropna()

    return grouped_df.reset_index(drop=True)

In [39]:
def group_same_songs(input_df, interval_threshold, duration_threshold):
    
    df = input_df.copy()

    df['start'] = pd.to_datetime(df['start'])
    df['end'] = pd.to_datetime(df['end'])

    df['group'] = 0

    for i in range(1, len(df)):
        if (df['start'][i] - df['end'][i - 1]).total_seconds() <= interval_threshold:
            df.at[i, 'group'] = df.at[i - 1, 'group']
        else:
            df.at[i, 'group'] = df.at[i - 1, 'group'] + 1

    grouped_df = df.groupby('group')

    for group, group_df in grouped_df:
        duration = (group_df['end'].max() - group_df['start'].min()).total_seconds()
        if duration < duration_threshold:
            df = df[df['group'] != group]

    df = df.drop(columns=['class_value'])

    df = df.groupby('group').agg({'start':'first', 'end':'last'})

    df['start'] = df['start'].apply(lambda x:x.strftime('%H:%M:%S'))
    df['end'] = df['end'].apply(lambda x:x.strftime('%H:%M:%S'))
    
    return df.reset_index(drop=True)

In [ ]:
def mfcc_from_accompanies(accompanies_paths, slice_duration=5):

    sliced_samples, sampling_rates = load_and_slice_audio(accompanies_paths, slice_duration)
    monofied_samples = [torch.mean(x, dim=0) for x in sliced_samples]
    sampling_rate = get_common_sampling_rate(sampling_rates)
    mfccs = [librosa.feature.mfcc(y=y.numpy(), sr=sampling_rate, n_mfcc=13) 
             for y in monofied_samples]
    
    return mfccs

In [167]:
def cluster_mfccs(mfccs):
    full_flattens = [x.flatten() for x in mfccs]
    kmeans = KMeans(n_clusters=2, n_init=2)  # You can choose the number of clusters
    kmeans.fit(full_flattens)
    cluster_labels = kmeans.labels_

    indices = np.arange(len(cluster_labels))
    temp_df = create_dataframe(np.vstack((indices, cluster_labels)).T, time_window_length=5)

    value_counts = temp_df['class'].value_counts()
    major_value = value_counts.idxmax()
    minor_value = value_counts.idxmin()

    label_mapping = {major_value: 'Speech', minor_value: 'Singing'}
    temp_df['class_value'] = temp_df['class'].map(label_mapping)

    return temp_df[temp_df['class_value']=='Singing'].reset_index(drop=True)

In [158]:
def seconds_to_hhmmss(total_seconds):
    m, s = divmod(total_seconds, 60)
    h, m = divmod(m, 60)
    return f"{h:02d}:{m:02d}:{s:02d}"
        
def hhmmss_to_seconds(hms):
    h, m, s = map(int, hms.split(':'))
    return h * 3600 + m * 60 + s

In [ ]:
def check_overlap(interval1_start, interval1_end, interval2_start, interval2_end):
    return (interval1_start <= interval2_end and interval1_end >= interval2_start) or \
           (interval2_start <= interval1_end and interval2_end >= interval1_start)

def find_and_filter_overlapping_timestamps(accompanies_df, vocal_df):
    unique_overlapping_timestamps = []

    for _, row1 in accompanies_df.iterrows():
        for _, row2 in vocal_df.iterrows():
            start1_seconds = hhmmss_to_seconds(row1['start'])
            end1_seconds = hhmmss_to_seconds(row1['end'])
            start2_seconds = hhmmss_to_seconds(row2['start'])
            end2_seconds = hhmmss_to_seconds(row2['end'])

            # Check for overlap
            if check_overlap(start1_seconds, end1_seconds, start2_seconds, end2_seconds):
                overlap_start = min(start1_seconds, start2_seconds)
                overlap_end = max(end1_seconds, end2_seconds)
                overlapping_timestamp = {
                    'start': seconds_to_hhmmss(overlap_start),
                    'end': seconds_to_hhmmss(overlap_end)
                }
                if overlapping_timestamp not in unique_overlapping_timestamps:
                    unique_overlapping_timestamps.append(overlapping_timestamp)

    # Filter out redundant overlaps
    filtered_overlapping_timestamps = [
        overlap for overlap in unique_overlapping_timestamps
        if all(
            not (
                hhmmss_to_seconds(overlap['start']) >= hhmmss_to_seconds(other_overlap['start']) and
                hhmmss_to_seconds(overlap['end']) <= hhmmss_to_seconds(other_overlap['end'])
            )
            for other_overlap in unique_overlapping_timestamps
            if overlap != other_overlap
        )
    ]

    merged_result = pd.DataFrame(filtered_overlapping_timestamps)

    return merged_result

In [ ]:
!rm -rf ./output

output_filename_pattern = 'noa'
root_dir = '.'  
youtube_url = 'https://www.youtube.com/watch?v=uvuDBBDUnEs'

separator = Separator("spleeter:2stems")

slice_wav_by_length(download_youtube_audio(youtube_url), output_filename_pattern)

wav_slice_paths = get_filepaths_with_string_and_extension(
    root_dir, target_string = output_filename_pattern, extension='wav'
)

seperate_vocal_and_accompanies(separator, wav_slice_paths)

accompanies_paths = get_filepaths_with_string_and_extension(
    root_dir, target_string = 'accompaniment', extension='wav'
)

vocal_paths = get_filepaths_with_string_and_extension(
    root_dir, target_string = 'vocal', extension='wav'
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = AutoModelForAudioClassification.from_pretrained(
    "MIT/ast-finetuned-audioset-10-10-0.4593"
).to(device)

In [25]:
result_vocal, raw_outputs = audio_analyzer(
    vocal_paths, model, device, slice_duration=5
)

  0%|          | 0/8 [00:00<?, ?it/s]

Processing samples:   0%|          | 0/840 [00:00<?, ?it/s]

In [26]:
classes = model.config.id2label
singing_like_indices = [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 112, 113, 114, 116, 117, 118, 119, 120, 136, 254, 255, 266, 259, 268, 269, 271, 402, 496, 503,523, 526]
singing_like_classes = [classes.get(x) for x in singing_like_indices]

In [296]:
vocal_result = group_same_songs(
    class_manipulator(result_vocal, singing_like_classes),
    interval_threshold=10, duration_threshold=30
)

In [254]:
mfccs = mfcc_from_accompanies(accompanies_paths, slice_duration=5)
accompanies_based_cluster = cluster_mfccs(mfccs)

accompanies_result = group_same_songs(
    accompanies_based_cluster,
    interval_threshold=10, duration_threshold=30
)

  0%|          | 0/8 [00:00<?, ?it/s]

In [310]:
higher_probability_timestamps = find_and_filter_overlapping_timestamps(accompanies_result, vocal_result)
print(higher_probability_timestamps.to_json())

      start       end
0  00:09:50  00:14:20
1  00:19:30  00:24:40
2  00:28:20  00:32:55
3  00:34:35  00:38:55
4  00:41:00  00:46:05
5  00:48:15  00:53:15
6  00:57:05  00:59:45


In [312]:
non_overlapping_rows = []
for index, row in vocal_result.iterrows():
    interval = {'start': row['start'], 'end': row['end']}
    overlaps = False
    for _, hp_row in higher_probability_timestamps.iterrows():
        hp_interval = {'start': hp_row['start'], 'end': hp_row['end']}
        if overlap(interval, hp_interval):
            overlaps = True
            break
    if not overlaps:
        non_overlapping_rows.append(row)
        
possible_a_capella = pd.DataFrame(non_overlapping_rows).reset_index(drop=True)

print(possible_a_capella.to_json())

{"start":{"0":"01:07:45"},"end":{"0":"01:09:05"}}
